## Övning 6
__John Landeholt__

johnlan@kth.se


__agenda:__

* komprimering
* kryptering

### Komprimering

Det finns två sorters komprimering:

* non-lossy compression - konverterar data
* lossy compression - tar bort data

__Följdlängdskodning__

Är en av de mer intuitiva kompressionsalgoritmerna, som minimerar följder av samma tecken genom att eliminera alla dubbletter och istället spara antalet som en siffra.

Men är inte så värst användbar i verkligenheten, då texter inte brukar innehålla långa följder av samma tecken.

In [1]:
from sys import getsizeof

def comp_rate(text,encoding):
    before = getsizeof(text)
    after = getsizeof(encoding)
    return (before - after) / after * 100

def rle_encoding(text):
    encoding = ''
    prev = ''
    count = 1
    for c in text:
        if c != prev:
            if prev:
                if count == 1:
                    encoding += prev
                else:
                    encoding += str(count) + prev
            count = 1
            prev = c
        else:
            count += 1
    encoding += str(count) + prev
    
    
    rate = comp_rate(text,encoding)
    print(f'compressed input with {round(rate, 2)}%')
    return encoding

encoding = rle_encoding('ÅÅÅÅH! JAAAAAAA! AAAAAAAAAAAAH.')
print(encoding)

In [2]:
def rle_decoding(encoding):
    decoding = ''
    count = ''
    for c in encoding:
        if c.isnumeric():
            count += c
        else:
            if count == '':
                decoding += c
            else:
                decoding += int(count) * c
                count = ''
    return decoding
text = rle_decoding(encoding)
print(text)

__Huffmankodning__

<img src="img/huffman.png" style="float:right" />

Går ut på att __räkna__ hur vanliga tecken är i en text. Där det tecknet med högst frekvens får kortast binärkod.

Man börjar med att räkna frekvensen av alla tecken i texten, sedan placerar du det i en prioritetskö, där den med högst prioritet har minst sannolikhet (en min-heap).

Sedan för varje par i kön så samlar du alla __0__ för det vänstra paret och __1__ för det högra paret, tills det endast finns ett par i kön, vilket kommer att vara roten som har sannolikheten __1__.

Detta innebär att du alltså börjar med löven i trädet och bygger dig upp för varje par.

På bilden ser vi att paret __(R,G)__ summeras upp till __2__ för att sedan i nästa loop bli till paret __(I, (R,G))__ vars summa är __4__ osv..

Sedan traversar man trädet igen och plockar med sig kodningen, som för __G__ blir __0111__

> __notera__: huffmankodning är endast non-lossy om `frekvenstabellen` sparas. Annars är den lossy.

In [2]:
from shared import Min_heap, Huffman_node
from collections import defaultdict
from time import sleep

def print_heap(heap):
    for n in heap.array[1:]:
        print(n, end= ' ')
    print()
        
def huffman_encode(text):
    freq = defaultdict(int)
    heap = Min_heap()
    
    for c in text: freq[c] += 1
    for k,v in freq.items(): heap.insert(Huffman_node(k,v))
    while len(heap) > 1:
        print_heap(heap)
        left, right = heap.pop(), heap.pop()
        for p in left.pairs: p += '0'
        for p in right.pairs: p += '1'
        n = Huffman_node.merge(left, right)
        heap.insert(n)
        sleep(3)
    print_heap(heap)
    encoding = ''
    node = heap.pop()
    for c in text:
        code = node[c]
        if code:
            encoding += code + ' '
    return encoding[:-1], freq

encoding, freq = huffman_encode('HAHA!IIIIIIH!AHRG...')
print(encoding)

{G: None}: 1 {!: None}: 2 {R: None}: 1 {I: None}: 6 {H: None}: 4 {A: None}: 3 {.: None}: 3 
{G: 0, R: 1}: 2 {.: None}: 3 {!: None}: 2 {I: None}: 6 {H: None}: 4 {A: None}: 3 
{A: None}: 3 {.: None}: 3 {H: None}: 4 {I: None}: 6 {G: 00, R: 01, !: 1}: 4 
{H: None}: 4 {G: 00, R: 01, !: 1}: 4 {I: None}: 6 {A: 0, .: 1}: 6 
{A: 0, .: 1}: 6 {I: None}: 6 {H: 0, G: 100, R: 101, !: 11}: 8 
{H: 0, G: 100, R: 101, !: 11}: 8 {A: 00, .: 01, I: 1}: 12 
{H: 00, G: 0100, R: 0101, !: 011, A: 100, .: 101, I: 11}: 20 
00 100 00 100 011 11 11 11 11 11 11 00 011 100 00 0101 0100 101 101 101


In [3]:
from shared import huffman_decode
huffman_decode(encoding, freq)

'HAHA!IIIIIIH!AHRG...'

### Huffmankodning

__Hur gör vi det för hand?__

Vi prövar med stringen "man är mans gamman".

Steg:

1. Beräkna frekvensen för alla tecken
2. Sortera så att det tecknet med minst frekvens är först
3. Placera dem i en kö så att det är minst -> störst
4. poppa kön 2 gånger, summera frekvenserna för paret ge första tecknet en __0__ och andra en __1__
5. pusha paret in i kön igen
6. repetera punkt 4-5 tills det endast finns en nod i kön.